In [115]:
from tempfile import tempdir

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import numpy as np
from torch.utils.data import DataLoader
import sys
device = torch.device("cuda")

In [116]:
# 导入图片
transform = transforms.Compose([
    transforms.Resize((224, 224)),# 随机crop后resize
    transforms.RandomRotation(20), # 随机旋转20度
    transforms.RandomHorizontalFlip(p=0.5), # 随机水平翻转
    transforms.ToTensor(),])

root = 'dataset'
train_dataset = datasets.ImageFolder(root+'/train', transform=transform)
test_dataset = datasets.ImageFolder(root+'/test', transform=transform)

size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=size, shuffle=True, pin_memory=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=size, shuffle=True, pin_memory=True,num_workers=4)

In [117]:
classes = train_dataset.classes
class_to_idx = train_dataset.class_to_idx
print(class_to_idx)

{'cat': 0, 'dog': 1}


In [118]:
# 载入已经训练好的官方模型
model = models.vgg16(pretrained=True)
model.to(device)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [119]:
# 循环模型所有参数，不计算梯度，不参与梯度更新。（卷积层不参与训练）
# for param in model.parameters():
#     param.requires_grad = False
    
# 为什么是25088，
# 初始输入：224x224x3
# 第一次最大池化后：112x112x64
# 第二次最大池化后：56x56x128
# 第三次最大池化后：28x28x256
# 第四次最大池化后：14x14x512
# 第五次最大池化后：7x7x512
model.classifier = nn.Sequential(nn.Linear(25088, 100), 
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(100, 2),
                                 ).to(device)

In [120]:
LR = 0.0003
entropy_loss = nn.CrossEntropyLoss()
entropy_loss.to(device)

optimizer = optim.Adam(model.parameters(), lr=LR)

In [121]:
def train():
    model.train()
    for i,data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = entropy_loss(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
def test():
    model.eval()
    correct = 0
    for i,data in enumerate(test_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    print("TestACC:{0}".format(correct/len(test_dataset)))
    
    correct = 0
    for i,data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    print("TrainACC:{0}".format(correct/len(train_dataset)))
        
    
        

In [123]:
for epoch in range(0,3):
    print('Epoch {}/{}'.format(epoch,3))
    train()
    test()
    
torch.save(model.state_dict(), 'catdog.pth')

Epoch 0/6
TestACC:0.8560000658035278
TrainACC:0.9864999651908875
Epoch 1/6
TestACC:0.8830000162124634
TrainACC:0.9928333163261414
Epoch 2/6
TestACC:0.8510000109672546
TrainACC:0.9788333177566528


# 验证，此部分单独运行

In [125]:
from PIL import  Image
import matplotlib.pyplot as plt

modeleval = models.vgg16(pretrained=True)
modeleval.classifier = nn.Sequential(nn.Linear(25088, 100), 
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(100, 2),
                                 )

modeleval.load_state_dict(torch.load('catdog.pth'))

modeleval.eval()

C:\Users\BLKDASH\AppData\Local\Temp\ipykernel_8496\1387754148.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modeleval.load_state_dict(torch.load('catdog.pth'))


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [133]:
# 定义标签
label = ['cat','dog']

# 预处理器
transformeval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    ])


In [139]:
def predict(img_path):
    img = Image.open(img_path)
    # 预处理，并增加一个维度
    img = transformeval(img).unsqueeze(0)
    output = modeleval(img)
    # 获取最大值位置
    _, predicted = torch.max(output.data, 1)
    # 转化为类别名称
    predicted_label = label[predicted.item()]
    
    # 显示图片
    # plt.imshow(img)
    # plt.axis('off')
    # plt.title(f'Predicted: {predicted_label}')
    # plt.show()
    print(predicted_label)

In [143]:
predict('dataset/cat/14334.jpg') #应该会输出猫
predict('dataset/cat/14335.jpg') #应该会输出猫
predict('dataset/dog/14336.jpg')
predict('dataset/dog/14337.jpg')
predict('dataset/cat/14338.jpg')
predict('dataset/dog/14339.jpg')

cat
cat
dog
dog
cat
dog
